In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/openofficepreprocessed6/openoffice_preprocessed.csv


In [2]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/tools/tokenization.py

In [3]:
!pip install sentencepiece

In [4]:
!pip install tensorflow_hub

In [5]:
!pip install torch

In [6]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download("words")
from nltk.corpus import stopwords
from nltk.corpus import words
# words = set(nltk.corpus.words.words())

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:
import pandas as pd
import numpy as np
from random import seed
from random import randint
import random
import string
import re
import matplotlib.pyplot as plt
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
from tensorflow.keras.layers import Dense, Input, Reshape, Conv1D, Conv2D, BatchNormalization, MaxPooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split

# from bert import bert_tokenization
import tokenization
from sklearn.model_selection import StratifiedKFold

In [9]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

2022-03-08 04:29:45.010598: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-08 04:29:45.059260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:29:45.158298: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:29:45.159498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [10]:
seed(1)
stop_words = stopwords.words('english')
words=words.words()

In [11]:
dataset = pd.read_csv('../input/openofficepreprocessed6/openoffice_preprocessed.csv')
dataset

,Unnamed: 0,bug_id,description1,short_desc1,dup_id,resolution,product,component,description2,short_desc2,duplicate
0,76733,92930,I am using the OpenOffice.org 2.4.1 suite on a...,Dialog boxes on 'off' monitor after switching ...,42282,DUPLICATE,Writer,ui,37085 I'm using OpenOffice 1.1.4 on Windows...,37085 Dualhead: popup dialogs appear off sc...,1
1,48083,55933,In the German version of OOo2.0_RC2 there's an...,[de] wrong ext. tooltip translation in toolbar...,52161,DUPLICATE,Internationalization,ui,"45216 Hi ufi,\n\nin m118 in the picture too...",45216 wrong text in extended tooltip of Col...,1
2,28488,32421,I tried having an OOo1.1.1 under WindowsXP and...,Sharing macros between Win/Linux version impos...,21151,DUPLICATE,General,code,19060 Product Requirement \nPlatform indepe...,19060 Q-PCD ConfigAdminDeploy-45: Arch inde...,1
3,56773,67117,If you insert an automatic date (current date;...,automatic dates field problem between FR and E...,29646,DUPLICATE,Writer,open-import,26113 Creating some date field under format...,26113 WW8: some foreign date formats conver...,1
4,40428,46276,OpenOffice DOC document Heap Overflow\n[Securi...,OpenOffice DOC document Heap Overflow importin...,46388,DUPLICATE,Writer,code,40520 OpenOffice DOC document Heap Overflow...,40520 OpenOffice DOC document Heap Overflow,1
...,...,...,...,...,...,...,...,...,...,...,...
100109,63603,75857,Attached is locale data for uz_UZ. Please inte...,Add Uzbek (uz_UZ) locale data,[],FIXED,Internationalization,localedata,test for reopening,test,0
100110,68481,82059,File will be sent to es@openoffice because it ...,"Some files produce ""General Input/Output error...",[],FIXED,Writer,open-import,When i translated this file in french and when...,Error when calc save a file .sxc,0
100111,89314,110009,"i.e. do what the comment says... ""after src680...",linguistic: replace StaticWithInit_ with rtl::...,[],FIXED,Writer,code,Because X Font are store in iso2022 encoding\n...,Patch to Draw Chiense GB2312 Character Correctly,0
100112,74911,90578,"Hello,\nI got error 504 using the SUMIF functi...",SUMIF function issue: error 504,[],NOT_AN_ISSUE,Calc,programming,As the current developpment for the coming OOo...,"Issuezilla ""Found in version"" is missing tag 644",0


In [12]:
dataset.dropna(subset = ["description2"], inplace=True)

In [13]:
dataset.dropna(subset = ["short_desc2"], inplace=True)

In [14]:
# dataset.loc[dataset['bug_id'] == 15957]

In [15]:
for index, row in dataset.iterrows():
    doc1 = 'Product:' + row['product'] + '.Component:' + row['component'] + '.Description:' + row['description1'] #+ '.Summary:' + row['short_desc1']
#     print(row['bug_id'])
#     print(row['description2'])
    doc2 = 'Product:' + row['product'] + '.Component:' + row['component'] + '.Description:' + row['description2'] #+ '.Summary:' + row['short_desc2']
    dataset.loc[index, 'doc1'] = doc1
    dataset.loc[index, 'doc2'] = doc2

In [16]:
for index, row in dataset.iterrows():
    desc = row['doc1'] + row['doc2']
    dataset.loc[index, 'description'] = desc

In [17]:
def reset_column_names():
  dataset.drop('description', axis=1, inplace=True)
  dataset.drop('short_desc1', axis=1, inplace=True)
  dataset.drop('description2', axis=1, inplace=True)
  dataset.drop('short_desc2', axis=1, inplace=True)

  dataset.rename(columns={'description_clean':'description'}, inplace=True) #,'short_desc1_clean':'short_desc1','description2_clean':'description2','short_desc2_clean':'short_desc2'

In [18]:
def preprocess_text_content(text):
    text = text.lower()
#     cleaned_text = re.sub(r"""
#                [,.;:@#?!&$*+-=_%<>\/\[\]\(\)\{\}\"\'\n]+  # Accept one or more copies of punctuation
#                \ *           # plus zero or more copies of a space,
#                """,
#                " ",          # and replace it with a single space
#                text, flags=re.VERBOSE)
#     cleaned_text = re.sub(r"https?:\/\/[A-Za-z0-9.\/?&#+*+-=_%]+", " ",text) #replace urls
    cleaned_text = re.sub(r" +", " ", text) #remove extra white spaces
#     cleaned_text = re.sub(r"[d|D]escription", "", cleaned_text)
    
    return cleaned_text

In [19]:
dataset['description_clean']= dataset['description'].apply(lambda x:preprocess_text_content(x))
# dataset['short_desc1_clean']= dataset['short_desc1'].apply(lambda x:preprocess_text(x))
# dataset['description2_clean']= dataset['description2'].apply(lambda x:preprocess_text(x))
# dataset['short_desc2_clean']= dataset['short_desc2'].apply(lambda x:preprocess_text(x))
dataset.head()

,Unnamed: 0,bug_id,description1,short_desc1,dup_id,resolution,product,component,description2,short_desc2,duplicate,doc1,doc2,description,description_clean
0,76733,92930,I am using the OpenOffice.org 2.4.1 suite on a...,Dialog boxes on 'off' monitor after switching ...,42282,DUPLICATE,Writer,ui,37085 I'm using OpenOffice 1.1.4 on Windows...,37085 Dualhead: popup dialogs appear off sc...,1,Product:Writer.Component:ui.Description:I am u...,Product:Writer.Component:ui.Description:37085 ...,Product:Writer.Component:ui.Description:I am u...,product:writer.component:ui.description:i am u...
1,48083,55933,In the German version of OOo2.0_RC2 there's an...,[de] wrong ext. tooltip translation in toolbar...,52161,DUPLICATE,Internationalization,ui,"45216 Hi ufi,\n\nin m118 in the picture too...",45216 wrong text in extended tooltip of Col...,1,Product:Internationalization.Component:ui.Desc...,Product:Internationalization.Component:ui.Desc...,Product:Internationalization.Component:ui.Desc...,product:internationalization.component:ui.desc...
2,28488,32421,I tried having an OOo1.1.1 under WindowsXP and...,Sharing macros between Win/Linux version impos...,21151,DUPLICATE,General,code,19060 Product Requirement \nPlatform indepe...,19060 Q-PCD ConfigAdminDeploy-45: Arch inde...,1,Product:General.Component:code.Description:I t...,Product:General.Component:code.Description:190...,Product:General.Component:code.Description:I t...,product:general.component:code.description:i t...
3,56773,67117,If you insert an automatic date (current date;...,automatic dates field problem between FR and E...,29646,DUPLICATE,Writer,open-import,26113 Creating some date field under format...,26113 WW8: some foreign date formats conver...,1,Product:Writer.Component:open-import.Descripti...,Product:Writer.Component:open-import.Descripti...,Product:Writer.Component:open-import.Descripti...,product:writer.component:open-import.descripti...
4,40428,46276,OpenOffice DOC document Heap Overflow\n[Securi...,OpenOffice DOC document Heap Overflow importin...,46388,DUPLICATE,Writer,code,40520 OpenOffice DOC document Heap Overflow...,40520 OpenOffice DOC document Heap Overflow,1,Product:Writer.Component:code.Description:Open...,Product:Writer.Component:code.Description:4052...,Product:Writer.Component:code.Description:Open...,product:writer.component:code.description:open...


In [20]:
reset_column_names()

In [21]:
# def preprocess_text(text,tokenizer):
# #     print(text)
#     #tokenize the text
#     text = tokenizer.tokenize(text)
#     #lowercase the text
#     text = [w.lower() for w in text]
#     #removal of stop word 
#     text = [w for w in text if w not in stop_words]
#     #removal of non-english words
#     text = [w for w in text if w in words or w.isalpha()]
    #stemming
#     text = [ ps.stem(w) for w in text]
    #remove extra white spaces
#     text = [ re.sub(r" +", " ", w) for w in text] 
#     cleaned_text = re.sub(r"""
#                [,.;:@#?!&$*+-=_%<>\/\[\]\(\)\{\}\"\'\n]+  # Accept one or more copies of punctuation
#                \ *           # plus zero or more copies of a space,
#                """,
#                " ",          # and replace it with a single space
#                text, flags=re.VERBOSE)
#     cleaned_text = re.sub(r"https?:\/\/[A-Za-z0-9.\/?&#+*+-=_%]+", " ",cleaned_text) #replace urls
    
#     cleaned_text = re.sub(r"[d|D]escription", "", cleaned_text)
    
#     return text

In [22]:
# max_len = 0
# lenArr = []
# outliers = []

# q25, q75 = np.percentile(lenArr, [25, 75])
# bin_width = 2 * (q75 - q25) * len(lenArr) ** (-1/3)
# bins = round((lenArr.max() - lenArr.min()) / bin_width)

### Add tokens to the data make it BERT compatible
def bert_encode(texts, tokenizer, max_len=512):
    print('encode')
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
#         text = preprocess_text(text,tokenizer)
        text = tokenizer.tokenize(text)
#         print(text)
        text = [w for w in text if w.lower() not in stop_words]
    
#         length = len(text)
#         if(length <500):
#             lenArr.append(length)
#     #     print(length)
#             if(max_len < length):
#               max_len = length
#         else:
#             outliers.append(length)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
#     print(len(lenArr))
#     # lenArr=list(filter(lambda a: a != 3395, lenArr))
#     # print(lenArr)
#     print("avg", sum(lenArr)/len(lenArr))
#     print("outliers", outliers)
#     print("outliers", len(outliers))
#     plt.hist(lenArr, density=True, bins=30)
#     max_len

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [23]:
def build_model(bert_layer, max_len=512):
    print('build model')
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
#     embedding = bert_layer(
#         input_word_ids, token_type_ids=segment_ids, attention_mask=input_mask
#     )[0]

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :] 
#     print(clf_output)
   
#     clf_output = Reshape((1024,1))(clf_output)
#     print(clf_output)
    
#     x = Conv1D(8, 3, activation='relu', padding='same')(clf_output)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D()(x)
#     x = Conv1D(32, 3, activation='relu', padding='same')(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D()(x)
#     x = Flatten()(x)
#     out = Dense(1, activation='sigmoid')(x)
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model

In [24]:
train, test = train_test_split(dataset, test_size=0.3)

In [25]:
FullTokenizer = tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1", trainable=True)

2022-03-08 04:41:43.759684: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:41:43.760590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:41:43.761237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:41:43.762154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-08 04:41:43.762789: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [26]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
lower_cased = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = FullTokenizer(vocab_file, lower_cased)

In [27]:
train_input = bert_encode(train['description'], tokenizer, max_len=160)
test_input = bert_encode(test['description'], tokenizer, max_len=160)
train_labels = np.array(train.duplicate.values, dtype='int')
test_labels = np.array(test.duplicate.values, dtype='int')

encode
encode


In [28]:
model = build_model(bert_layer, max_len=160)
model.summary()

build model
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0] 

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [29]:
model.fit(train_input,train_labels, epochs=2, batch_size=15)

2022-03-08 04:52:12.546796: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2
4672/4672 [==============================] - 4143s 880ms/step - loss: 0.1286 - accuracy: 0.9301 - precision: 0.9256 - recall: 0.9352
Epoch 2/2
4672/4672 [==============================] - 4110s 880ms/step - loss: 0.0515 - accuracy: 0.9788 - precision: 0.9764 - recall: 0.9812


In [30]:
model.save("openoffice-bert-model")

2022-03-08 07:10:14.244628: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [31]:
evaluation_results = model.evaluate(test_input,test_labels,return_dict=True)
evaluation_results

939/939 [==============================] - 567s 600ms/step - loss: 0.0387 - accuracy: 0.9866 - precision: 0.9846 - recall: 0.9888


{'loss': 0.03873603045940399,
 'accuracy': 0.9866467118263245,
 'precision': 0.9845787286758423,
 'recall': 0.9888327717781067}

In [32]:
# X = dataset['description']
# Y = np.array(dataset.duplicate.values, dtype='int')

# kfold = StratifiedKFold()
# cvscores = []
# precision_scores = []
# recall_scores = []
# f1_scores = []
    
# for train, test in kfold.split(X, Y):
#     # Prepare data
#     train_input = bert_encode(X.iloc[train], tokenizer, max_len=160)
#     test_input = bert_encode(X.iloc[test], tokenizer, max_len=160)
#     train_labels = Y[train]
#     test_labels = Y[test]
    
#     model = build_model(bert_layer, max_len=160)
#     model.summary()

#     # Fit the model
#     model.fit(train_input,train_labels, epochs=1, batch_size=15)

#     # Evaluate the model
#     scores = model.evaluate(test_input, test_labels, verbose=0)
#     f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
#     print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#     print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
#     print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
#     print("f1 score: ", (f1score))
#     cvscores.append(scores[1] * 100)
#     precision_scores.append(scores[2] * 100)
#     recall_scores.append(scores[3] * 100)
#     f1_scores.append(f1score)
    

# print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
# print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision_scores), np.std(precision_scores)))
# print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall_scores), np.std(recall_scores)))
# print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))